# Running gridded DBPM run
**Author**: Denisse Fierro Arcos  
**Date**: 2025-01-13  

Once all necessary inputs have been processed and fishing parameters calculated, we can finally run the gridded version of DBPM.  

Note that the temporal range of DBPM inputs needs to be taken into account before running the model.

## Loading relevant libraries

In [1]:
import os
os.chdir('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow/')
from glob import glob
import xarray as xr
import useful_functions as uf
from dask.distributed import Client

## Start a cluster for parallelisation

In [2]:
client = Client(threads_per_worker = 1)

2025-01-13 15:25:09,761 - distributed.scheduler - WARNING - Detected different `run_spec` for key ('original-open_dataset-size_bins-72c7f0fba951651fb99d2c971bf417b8', 0) between two consecutive calls to `update_graph`. This can cause failures and deadlocks down the line. Please ensure unique key names. If you are using a standard dask collections, consider releasing all the data before resubmitting another computation. More details and help can be found at https://github.com/dask/dask/issues/9888. 
Debugging information
---------------------
old task state: released
old run_spec: <Task ('original-open_dataset-size_bins-72c7f0fba951651fb99d2c971bf417b8', 0) _execute_subgraph(...)>
new run_spec: <Task ('original-open_dataset-size_bins-72c7f0fba951651fb99d2c971bf417b8', 0) _execute_subgraph(...)>
old token: '362a2827097e72747b479dab2e477614'
new token: '2a06d6e4001fc62ec8acd727236b2a4c'
old dependencies: set()
new dependencies: set()

2025-01-13 15:25:40,561 - distributed.scheduler - WARN

## Defining input and output folders

In [2]:
base_folder = '/g/data/vf71/la6889/dbpm_inputs/east_antarctica/gridded_params/'
out_folder = '/g/data/vf71/la6889/dbpm_inputs/east_antarctica/run_fishing'
os.makedirs(out_folder, exist_ok = True)

## Loading fixed DBPM parameters 

In [3]:
depth = xr.open_zarr(
     glob(os.path.join('/g/data/vf71/la6889/dbpm_inputs/east_antarctica/gridded', 
                      '*obsclim_deptho_*'))[0])['deptho']
log10_size_bins_mat = xr.open_zarr('outputs/log10_size_bins_matrix.zarr/')['size_bins']

## Loading dynamic DBPM parameters
Data for `predators`, `detritivores`, and `detritus` have an initialising timestep. This means that they start one month before all other parameters, but they end on the same date.

### `Predators`

In [4]:
# Loading initial predator data
predators = xr.open_zarr(glob(
    os.path.join(base_folder, 
                 'predators_spinup*1840*'))[0])['predators'] 

*Optional:* If DBPM could not be run for the entire decade covered by the original `predator` data, we can apply the following steps to start the model from any time step. **Remember**, you will need to ensure the `predator` data starts on time step before all other data.

In [5]:
#Load predator results for the month before you want to run DBPM
p = xr.open_dataarray(os.path.join(out_folder, 'predators_15arcmin_fao-58_1841-08.nc'))
#Subset predator data for the period you want to run the model
predators = predators.sel(time = slice('1841-09', '1849-12'))
#Combine both data arrays
predators = xr.concat([p, predators], dim = 'time')

### `Detritivores` and `detritus`

In [6]:
# detritivores = xr.open_zarr(glob(
#     os.path.join(base_folder, 
#                  'detritivores_*'))[0])['detritivores']

# detritus = xr.open_zarr(glob(
#     os.path.join(base_folder, 
#                  'detritus_*'))[0])['detritus']

*Optional:* If you are running DBPM from time than the start of the original `predator` data, you will need to load results for `detritivores` and `detritus` for the time step prior to the time step you want to start the model.

In [ ]:
detritivores = xr.open_dataarray(os.path.join(out_folder, 
                                              'detritivores_15arcmin_fao-58_1841-08.nc'))

detritus = xr.open_dataarray(os.path.join(out_folder,
                                          'detritus_15arcmin_fao-58_1841-08.nc'))

### Fishing `effort`

In [ ]:
effort = xr.open_zarr(
    glob(os.path.join(base_folder, 
                      'effort_spinup*'))[0])['effort']

*Optional:* If you are running DBPM from time than the start of the original `predator` data, you will need to subset the data from the time step after you want to start the model, and load the `effort` results for the time step you want to start the model from. 

In [ ]:
#Load effort for time step DBPM starts
e_start = xr.open_dataarray(os.path.join(out_folder, 'effort_15arcmin_fao-58_1841-09.nc'))
#Subset effort data from the month after DBPM begins
effort = effort.sel(time = slice('1841-10', '1849-12'))
#Combine both data arrays
effort = xr.concat([e_start, effort], dim = 'time')

### Temperature effects for pelagic and benthic organisms

In [8]:
pel_tempeffect = xr.open_zarr(glob(
    os.path.join(base_folder, 'pel-temp-eff_spinup*'))[0])['pel_temp_eff']

ben_tempeffect = xr.open_zarr(glob(
    os.path.join(base_folder, 'ben-temp-eff_spinup*'))[0])['ben_temp_eff']

*Optional:* If you are running DBPM from time than the start of the original `predator` data, you will need to subset the data from the time step you want to start the model.

In [9]:
pel_tempeffect = pel_tempeffect.sel(time = slice('1841-09', '1849-12'))
ben_tempeffect = ben_tempeffect.sel(time = slice('1841-09', '1849-12'))

### Sinking rate

In [11]:
sinking_rate = xr.open_zarr(
    glob(os.path.join('/g/data/vf71/la6889/dbpm_inputs/east_antarctica/gridded', 
                      '*_spinup_er_*'))[0])['export_ratio']

*Optional:* If you are running DBPM from time than the start of the original `predator` data, you will need to subset the data from the time step you want to start the model.

In [ ]:
sinking_rate = sinking_rate.sel(time = slice('1841-09', '1849-12'))

## Running gridded DBPM
Note that this may take several hours depending on computing resources, as well as the temporal and spatial range of the modelled area.

In [ ]:
uf.gridded_sizemodel(base_folder, predators, detritivores, detritus, pel_tempeffect, 
                     ben_tempeffect, effort, sinking_rate, depth, log10_size_bins_mat,
                     region = 'fao-58', out_folder = out_folder)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/dask/_task_spec.py:740: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/g/data/hh5/public/apps/minico

## Outside time loop

### Fisheries catches per year per size class
`fishing_mort_pred` and `fishing_mort_det` are set to zero outside the sizes for each class, so there is no need to apply a mask here.

In [356]:
catch_pred = fishing_mort_pred*predators*size_bin_vals
catch_det = fishing_mort_det*detritivores*size_bin_vals